In [1]:
import os
import json
import textwrap
import jsonlines
from collections import defaultdict
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

### General Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/General/Input/notes-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# load the service

info = json.load(open(os.environ['MICROSOFT_CREDENTIALS']))
credential = AzureKeyCredential(info['key'])
client = TextAnalyticsClient(endpoint=info['endpoint'], credential=credential)

In [4]:
# make predictions

width = 5120
outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    texts = textwrap.wrap(note, width, 
                          break_long_words=False, break_on_hyphens=False, 
                          replace_whitespace=False, drop_whitespace=False)
    responses = client.recognize_pii_entities(texts)
    
    size = 0
    for text, response in zip(texts, responses):
        for entity in response.entities:
            if entity.category == 'Person':
                outputs[ID][(size+entity.offset, size+entity.offset+entity.length)] = (entity.text, entity.confidence_score)
        size += len(text)
    
    if count % 100 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 1000
Finish Processing Note 2000
Finish Processing Note 3000
Finish Processing Note 4000
Finish Processing Note 5000
Finish Processing Note 6000
Finish Processing Note 7000
Finish Processing Note 8000
Finish Processing Note 9000
Finish Processing Note 10000
Finish Processing Note 11000
Finish Processing Note 12000
Finish Processing Note 13000
Finish Processing Note 14000
Finish Processing Note 15000


In [5]:
# save the outputs

with jsonlines.open('../Data/General/Output/notes-Microsoft.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])

### Polysemy Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/Polysemy/Input/polysemies-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# create a language resource to obtain the key and endpoint, install the azure client library
# load the service

info = json.load(open(os.environ['MICROSOFT_CREDENTIALS']))
credential = AzureKeyCredential(info['key'])
client = TextAnalyticsClient(endpoint=info['endpoint'], credential=credential)

In [4]:
# make predictions

width = 5120
outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    texts = textwrap.wrap(note, width, 
                          break_long_words=False, break_on_hyphens=False, 
                          replace_whitespace=False, drop_whitespace=False)
    responses = client.recognize_pii_entities(texts)
    
    size = 0
    for text, response in zip(texts, responses):
        for entity in response.entities:
            if entity.category == 'Person':
                outputs[ID][(size+entity.offset, size+entity.offset+entity.length)] = (entity.text, entity.confidence_score)
        size += len(text)
    
    if count % 100 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 100
Finish Processing Note 200


In [5]:
# save the outputs

with jsonlines.open('../Data/Polysemy/Output/polysemies-Microsoft.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])